# Data Tooling 101 Sandbox

### **What the heck is a Jupyter Notebook?**
A Jupyter Notebook is an interactive code environment that allows you to compartmentalize your code (and even markdown like this!!!) within the context of "cells". Code can be separated and executed in any order, on demand, within a specific cell's scope, but whatever gets executed affects a global scope, allowing you to create new global variables and update them in any order you choose.

### 1. Executing Cells out of Order

In [ ]:
# Execute this first
# Execute this third
try:
    print(an_uninstantiated_variable)
except:
    print("Your variable is uninstantiated!")

In [ ]:
# Execute this second
an_uninstantiated_variable = "I'm instantiated now!"

#### **Exercise**

Using what you know about Jupyter Notebooks, create a better implementation of the `get_fibonacci_term` implementation provided below as `get_fibonacci_term_faster`. Both functions MUST execute using the timeit decorator, side-by-side, so that the print statements show up next to each other.

Here are some important limitations of this exercise:
* You can only modify the cell that is marked "Exercise Solution"
* Both implementations must be called by the cell that is marked "Exercise Validation"
* You can otherwise execute these cells in any order

In [ ]:
def timeit(func):
    import time
    import traceback
    def wrapper(*args, **kwargs):
        before = time.time()
        result = func(*args, **kwargs)
        after = time.time()
        print(f"{func.__name__} took {after - before} seconds to finish.")
        return result
    return wrapper

@timeit
def get_fibonacci_term(n):
    def recursive(n):
        if n <= 2:
            return 1
        return recursive(n - 1) + recursive(n - 2)
    return recursive(n)

In [ ]:
funcs = [get_fibonacci_term]

In [ ]:
# Exercise Validation
for func in funcs:
    print(func(30))

In [ ]:
# Exercise Solution

### 2. Intro to NumPy and Pandas

NumPy (commonly aliased as `np`) and Pandas (commonly aliased as `pd`) are two of the most, if not the most, essential modules for doing data science work in Python. Consider learning them a hard requirement if you ever want to work with big data. That's NOT an exaggeration.

Python has many optimizations that make it much more performant than Javascript when it comes to array-like data structures. Javascript has strings, arrays, and objects. Similarly, Python has strings, lists, and dictionaries. (There are many more examples but these are the first three comparisons people think of). That being said, Python is a higher-level language and won't be nearly as performant as a Java, C, or C++ when it comes to raw optimization of very large datasets.

To address this, NumPy and Pandas leverage C and C++ under the hood in order to offload the computational grunt work onto those lower level languages. They also introduce three new and very important data structures that build off of our mental model of lists and dictionaries:

1. array (NumPy): A NumPy array is a single-to-multi dimensional container that holds elements of the same data type. NumPy arrays are used for efficient storage, manipulation, and computation of large amounts of numerical data.

2. Series (Pandas): A Pandas Series is a one-dimensional labeled data structure provided by the pandas library in Python. It's similar to a column in a spreadsheet or a one-column database table. A Series consists of an ordered sequence of values and associated index labels, which can be used for efficient data manipulation and analysis.

3. DataFrame (Pandas): A Pandas DataFrame is a two-dimensional labeled data structure provided by the pandas library in Python. It's akin to a table in a relational database or a spreadsheet, where data is organized into rows and columns. DataFrames are versatile and widely used for data manipulation, analysis, and exploration.

In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Generating a large list v. a large array of random numbers and multipling it by 2.
l = [random.random()*100 for _ in range(int(1e7))]
a = np.array(l)

@timeit
def multiply_by_n(d,n):
    if type(d) == list:
        return map(lambda x: x * 2, d)
    elif type(d) == np.array:
        return d * n
    return d
    
for d in [l,a]:
    multiply_by_n(d, 2)

In [ ]:
# Creating a dataframe from a list of objects
template = {
    'color': ['red','green','blue','orange','black','white','gray'],
    'car_type': ['compact','sedan','truck','suv','crossover','van','hatchback'],
    'fuel_type': ['gas','diesel','electric','hybrid','hydrogen'],
    'year': [2023,2022,2021,2020,2019,2018,2017],
    'price_range': [10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000],
    'condition': ['new', 'used'],
    'finance_options': ['cash','finance','lease'],
    'make': ['toyota','honda','ford','subaru','chevrolet','kia','gm']
}

def create_randomized_cars_dataset(n):
    dataset = []
    for _ in range(int(n)):
        d = {}
        for key in template.keys():
            d[key] = template[key][random.randint(0,len(template[key])-1)]
        dataset.append(d)
    
    return pd.DataFrame(dataset)

# head(n) gives us the first n rows of the dataframe
df = create_randomized_cars_dataset(1e4)
df.head(10)

In [ ]:
# A series would represent a single column or row of a dataframe
df.car_type

### 3. Applying ETL Processes to a CSV

In the data engineering world, a key acronym that describes the key components of data pipelining is ETL, or Extract, Transform, and Load.

1. Extract: Grabbing data from one or more sources of data (typically but not always external) to serve as the raw base data for you to change as part of a data pipeline.

2. Transform: How are we going to combine and process the data we've extracted so that it provides new value to our stakeholders? Common transformations might include filtering, merging, aggregating, and labeling, among others.

3. Load: Taking the data that we extracted and/or transformed and delivering it to some internal resource we own, that can be accessed by key stakeholders within our team, department, or organization.

ETL processes can be ad-hoc or periodic. They can be manually executed or scheduled. They could involve local machines, bare metal servers, or cloud-based resources. They can be in the form of databases, CSVs, or flat files.

In [ ]:
# Let's extract a sample dataset of cars data saved to this repo.
# We could generate this dynamically, but this static file will
# ensure that we get reproducible results.

path = '../data/sample_cars.csv'
cars = pd.read_csv(path)
cars.head(10)

#### 3.1: Filtering DataFrames

As the name implies, filtering allows you to view only a subset of your dataframe based on a set of logical conditions. The most popular way that folks use filtering is by a concept called "masking", where you apply a boolean operation to your DataFrame, which results in a DataFrame of Trues and/or Falses depending on whether the condition is matched. We then mask our original DataFrame based on that condition, and only those rows/columns with True will remain viewable.

As you get more comfortable working with DataFrames, there is also a `.query()` syntax that will allow you to pass in strings that have logical conditions in them that conform more to Python syntax, and which references columns directly by their names, and not via dot or square bracket notation.

In [ ]:
# If we want to have multiple filters applied as AND, use & between them
# If we want to have multiple filters applied as OR, use | between them
is_truck = cars.car_type == 'truck'

# If we want to sort values, we can use the .sort_values() method
# to list one or more columns to sort by, sequentially.
# The following sorting should provide black vehicles made in 2023 first, and white vehicles in 2017 last.
cars[is_truck].sort_values(by=["color", "year"], ascending=[True, False])

#### **Exercise**
Using what we learned about filters and masks, can you filter this dataset to only include cars that are under $20,000 or otherwise can be financed (not leased)?

In [ ]:
# Exercise Solution

#### Example 3.2: Transforming DataFrames

We can transform datasets in a number of ways in Pandas, but the main rule you want to remember is whether a Pandas method will transform a dataset by mutating it, or return a copy of a dataframe with a transformation. When in doubt, Pandas DOES have a `.copy()` method to DataFrames so that you can always have a clean copy to work with.

So what are some of the ways we can transform data in Pandas?
1. We can apply a vectorized operation on the dataframe itself for certain mathematical or simple logical operations by just writing the operation literally.
2. We can apply a change iteratively using `.apply()`, which is much slower because it's not vectorized. Alternatively, you can consider leveraging NumPy's `np.vectorize` on a callback function and then pass in the appropriate columns/rows that you would have accessed in the `.apply()` (See [this Medium article](https://michael-taverner.medium.com/stop-using-apply-and-start-using-numpys-vectorize-d589d15bc77b))
3. We can merge datasets horizontally (add new columns) using the `.merge()` method, which is Pandas' equivalent of a JOIN operation in SQL.
4. We can append datasets vertically (combine several DataFrames that have the same schema) using `.concat()` method, which is Pandas' equivalent of a UNION or UNION ALL operation in SQL.

**Testing out various element-wise operations**

In [ ]:
# Vectorized operation v. apply v. np.vectorize
@timeit
def apply_discount_math(df, pct_off=20, old_col="price_range", new_col="discounted_price_range"):
    new = df.copy()
    new[new_col] = new[old_col] * (100 - pct_off)/(100)
    return new

@timeit
def apply_discount_pandas_apply(df, pct_off=20, old_col="price_range", new_col="discounted_price_range"):
    new = df.copy()
    # What the hell is a lambda? It's an anonymous function! The JS equivalent
    # would be just creating in-line an arrow function or an unnamed function like
    # (x) => x[old_col] * (100 - pct_off)/(100) (remember, old_col and pct_off are parent-scoped)
    # x, combined with the axis = 1 represents a single row of the dataframe (kind of an object/dictionary)
    new[new_col] = new.apply(lambda x: x[old_col] * (100 - pct_off)/(100), axis=1)
    return new

@timeit
def apply_discount_numpy_vectorize(df, pct_off=20, old_col="price_range", new_col="discounted_price_range"):
    def apply_discount(old_price, pct_off):
        return old_price * (100 - pct_off)/(100)
    new = df.copy()
    apply_discount = np.vectorize(apply_discount)
    new[new_col] = apply_discount(new[old_col], pct_off)
    return new

In [ ]:
math = apply_discount_math(cars)
math.head(5)

In [ ]:
pd_apply = apply_discount_pandas_apply(cars)
pd_apply.head(5)

In [ ]:
np_vectorize = apply_discount_numpy_vectorize(cars)
np_vectorize.head(5)

#### Exercise

Now that you've seen the three different ways to create a new DataFrame column, can you create a new column, "price_range_after_financing", that will capture the following logic?
1. If the finance_option is "lease", add a 5% premium (so 105% of the cost) to the price_range
2. If the finance_option is "finance", add a 10% premium (so 110% of the cost) to the price_range
3. If the finance_option is "cash", keep the price_range the same since you're paying for it outright.

In [ ]:
# Exercise Solution

**Merging datasets**

Say that we have another dataset that represents paint vendors when you have to repair paint damage on the car, and that orange and green are premium colors that have to go to specific vendors, otherwise, the manufacturer will deal with them.

We can use a `.merge()` to join these two datasets together

In [ ]:
paint_vendors = pd.DataFrame({ "color": ["orange", "green"], "paint_vendor": ["Cuztom Colorz", "PNW Verdantry"]})
paint_vendors

In [ ]:
cars_with_vendors = cars.merge(paint_vendors, how="left", on="color")
cars_with_vendors

But in the above, you see we have a bunch of NaN results! How do we backfill them so that they show the manufacturer instead? There's another function, `.fillna()` that is helpful here!

In [ ]:
cars_with_vendors_clean = cars_with_vendors.copy()
cars_with_vendors_clean["paint_vendor"] = cars_with_vendors_clean["paint_vendor"].fillna(cars_with_vendors["make"])
cars_with_vendors_clean

#### Exercise

Now that you have a sense of how `.merge()` is used in Pandas, we want to merge this `cars` dataset with another dataset, `potential_buyers`, where a buyer has identified a specific interest in a single car_type for each row of data. Can we merge these two datasets and ONLY show the rows of data that actually have a match?

In [ ]:
potential_buyers = pd.DataFrame({
    "car_type": ["truck", "suv", "suv", "hatchback", "sedan", "sedan", "hatchback", "truck"],
    "buyer": ["Sam", "Sam", "Chris", "Alex", "Alex", "Dee", "Chris", "Dee"]
})
potential_buyers
# Exercise Solution

**Combining datasets through a concat**

Combining several Pandas DataFrames using a concat is dead-simple, assuming that the columns for each dataset match correctly, in sequence and with indexing.

In [ ]:
cars_5 = create_randomized_cars_dataset(5)
cars_5

In [ ]:
cars_10 = create_randomized_cars_dataset(10)
cars_10

In [ ]:
union = pd.concat([cars_5, cars_10], ignore_index=True)
union

#### Example 3.3: Group Bys and Aggregating Datasets

There are a ton of aggregation operations that are available via Pandas, and we won't go into much detail here, but things like count, sum, etc. are things that you can do across your entire dataset, or with grouping by particular column values or groups of column values using the `.groupby()` method before the aggregation you're attempting to run. If you're looking for a suite of statistical aggregations all at once (and your dataset is appropriate for them, you can also check out the `.describe()` method, which will give you min, max, standard deviations, counts, and percentile values. [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)

And, if you want to get even MORE generic with aggregations, check out the `.agg()` method, which lets you apply more general Python methods as your aggregation. [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)

In [ ]:
# Getting a count based on color
cars.groupby("color").count()

In [ ]:
# Getting the make based on color
cars.groupby("color")["make"].apply(lambda x: list(set(x)))

### 4. Let's play with survey data!

### 5. Visualizations with seaborn

TBD in Part 2 of this series!